# Hybrid Models Training

Training hybrid models:
- Model 3: Hybrid CNN + MLP
- Model 4: Multi-modal Fusion

In [2]:
import sys
from pathlib import Path
import torch
import torch.nn as nn
import json

PROJECT_ROOT = Path('/Volumes/SSanDisk/SpeechRec-German/without_context_windows')
sys.path.insert(0, str(PROJECT_ROOT))

from models.hybrid.hybrid_cnn_mlp import HybridCNNMLP
from models.hybrid.multimodal_fusion import MultiModalFusion
from utils.training_utils import train_model, evaluate_model
from utils.data_loader import load_data, create_dataloaders

if torch.backends.mps.is_available():
    device = torch.device("mps")
    print(f"Using MPS device")
elif torch.cuda.is_available():
    device = torch.device("cuda")
else:
    device = torch.device("cpu")

df, spectrograms_dict, feature_cols, feature_scaler, class_weights_dict = load_data(PROJECT_ROOT)
dataloaders = create_dataloaders(df, spectrograms_dict, feature_cols, feature_scaler, class_weights_dict, batch_size=64)

train_hybrid_loader = dataloaders['hybrid']['train']
val_hybrid_loader = dataloaders['hybrid']['val']
test_hybrid_loader = dataloaders['hybrid']['test']

OUTPUT_DIR = PROJECT_ROOT / 'artifacts' / 'b-p_dl_models'
class_weights = torch.tensor([class_weights_dict.get('0', class_weights_dict.get(0, 1.0)), 
                              class_weights_dict.get('1', class_weights_dict.get(1, 1.0))], dtype=torch.float32).to(device)

Using MPS device
Columns in df_phonemes: ['phoneme_id', 'utterance_id', 'phoneme', 'class', 'start_ms', 'end_ms', 'duration_ms', 'audio_path']
Columns in df_features: ['energy_rms', 'energy_rms_std', 'energy_zcr', 'energy_zcr_std', 'spectral_centroid', 'spectral_centroid_std', 'spectral_rolloff', 'spectral_rolloff_std', 'spectral_bandwidth', 'spectral_bandwidth_std', 'formant_f1', 'formant_f2', 'formant_f3', 'formant_f4', 'formant_f1_std', 'formant_f2_std', 'formant_f3_std', 'formant_f4_std', 'spectral_flatness', 'harmonic_noise_ratio', 'zcr_mean', 'energy_cv', 'phoneme_id', 'class', 'duration_ms', 'mfcc_mean_0', 'mfcc_mean_1', 'mfcc_mean_2', 'mfcc_mean_3', 'mfcc_mean_4', 'mfcc_mean_5', 'mfcc_mean_6', 'mfcc_mean_7', 'mfcc_mean_8', 'mfcc_mean_9', 'mfcc_mean_10', 'mfcc_mean_11', 'mfcc_mean_12', 'mfcc_std_0', 'mfcc_std_1', 'mfcc_std_2', 'mfcc_std_3', 'mfcc_std_4', 'mfcc_std_5', 'mfcc_std_6', 'mfcc_std_7', 'mfcc_std_8', 'mfcc_std_9', 'mfcc_std_10', 'mfcc_std_11', 'mfcc_std_12', 'delta_mfcc

## Model 3: Hybrid CNN + MLP

In [2]:
model3 = HybridCNNMLP(n_features=len(feature_cols), num_classes=2).to(device)
criterion = nn.CrossEntropyLoss(weight=class_weights)
optimizer = torch.optim.Adam(model3.parameters(), lr=1e-3)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=5)

save_dir = OUTPUT_DIR / 'hybrid_models' / 'hybrid_cnn_mlp'
save_dir.mkdir(parents=True, exist_ok=True)

history3, best_epoch3 = train_model(model3, train_hybrid_loader, val_hybrid_loader, criterion, optimizer, scheduler,
                                    device, num_epochs=50, save_dir=save_dir, model_name='hybrid_cnn_mlp', early_stopping_patience=10)

checkpoint = torch.load(save_dir / 'best_model.pt')
model3.load_state_dict(checkpoint['model_state_dict'])
test_metrics3, _, _, _ = evaluate_model(model3, test_hybrid_loader, criterion, device)

with open(save_dir / 'test_metrics.json', 'w') as f:
    json.dump(test_metrics3, f, indent=2)

print(f"Model 3 Test - Acc: {test_metrics3['accuracy']:.4f}, F1: {test_metrics3['f1']:.4f}, ROC-AUC: {test_metrics3['roc_auc']:.4f}")


Epoch 1/50
--------------------------------------------------


Train Loss: 0.2327, Train Acc: 0.8887
Val Loss: 0.2211, Val Acc: 0.8917
Val F1: 0.8950, Val ROC-AUC: 0.9782
✓ New best model saved! (F1: 0.8950)

Epoch 2/50
--------------------------------------------------


Train Loss: 0.1786, Train Acc: 0.9188
Val Loss: 0.1881, Val Acc: 0.9201
Val F1: 0.9218, Val ROC-AUC: 0.9803
✓ New best model saved! (F1: 0.9218)

Epoch 3/50
--------------------------------------------------


Train Loss: 0.1581, Train Acc: 0.9268
Val Loss: 0.1782, Val Acc: 0.9250
Val F1: 0.9263, Val ROC-AUC: 0.9807
✓ New best model saved! (F1: 0.9263)

Epoch 4/50
--------------------------------------------------


Train Loss: 0.1500, Train Acc: 0.9314
Val Loss: 0.1681, Val Acc: 0.9323
Val F1: 0.9333, Val ROC-AUC: 0.9828
✓ New best model saved! (F1: 0.9333)

Epoch 5/50
--------------------------------------------------


...

Train Loss: 0.0460, Train Acc: 0.9797
Val Loss: 0.1898, Val Acc: 0.9522
Val F1: 0.9524, Val ROC-AUC: 0.9871

Epoch 48/50
--------------------------------------------------


Train Loss: 0.0474, Train Acc: 0.9782
Val Loss: 0.1988, Val Acc: 0.9533
Val F1: 0.9535, Val ROC-AUC: 0.9872

Epoch 49/50
--------------------------------------------------


Train Loss: 0.0434, Train Acc: 0.9809
Val Loss: 0.1963, Val Acc: 0.9530
Val F1: 0.9530, Val ROC-AUC: 0.9872

Epoch 50/50
--------------------------------------------------


Train Loss: 0.0450, Train Acc: 0.9793
Val Loss: 0.1982, Val Acc: 0.9522
Val F1: 0.9523, Val ROC-AUC: 0.9871


Model 3 Test - Acc: 0.9478, F1: 0.9478, ROC-AUC: 0.9875


## Model 4: Multi-modal Fusion

In [3]:
# Need to create a combined dataloader for multi-modal (spectrogram + features + audio)
# For now, using hybrid dataloader and adding audio branch
from torch.utils.data import Dataset, DataLoader, WeightedRandomSampler
import numpy as np
import torch
from utils.data_loader import HybridDataset, RawAudioDataset

class MultiModalDataset(Dataset):
    def __init__(self, df, spectrograms_dict, feature_cols, scaler, split='train', audio_length=3200):
        self.hybrid_ds = HybridDataset(df, spectrograms_dict, feature_cols, scaler, split)
        self.audio_ds = RawAudioDataset(df, split=split, max_length=audio_length)
        
    def __len__(self):
        return len(self.hybrid_ds)
    
    def __getitem__(self, idx):
        (spec, feat), label = self.hybrid_ds[idx]
        audio, _ = self.audio_ds[idx]
        return (spec, feat, audio), label

def multimodal_collate_fn(batch):
    """Custom collate function for multimodal datasets that return ((spectrogram, features, audio), label)"""
    spectrograms = []
    features = []
    audios = []
    labels = []
    
    for item in batch:
        # item is ((spectrogram, features, audio), label)
        (spectrogram, feature, audio), label = item
        spectrograms.append(spectrogram)
        features.append(feature)
        audios.append(audio)
        labels.append(label)
    
    # Stack into batches
    spectrograms = torch.stack(spectrograms)
    features = torch.stack(features)
    audios = torch.stack(audios)
    labels = torch.stack(labels)
    
    return (spectrograms, features, audios), labels

train_mm_ds = MultiModalDataset(df, spectrograms_dict, feature_cols, feature_scaler, 'train')
val_mm_ds = MultiModalDataset(df, spectrograms_dict, feature_cols, feature_scaler, 'val')
test_mm_ds = MultiModalDataset(df, spectrograms_dict, feature_cols, feature_scaler, 'test')

train_labels = df[df['split'] == 'train']['class_encoded'].values
sample_weights = np.array([class_weights_dict.get(str(l), class_weights_dict.get(l, 1.0)) for l in train_labels])
sampler = WeightedRandomSampler(weights=sample_weights, num_samples=len(sample_weights), replacement=True)

train_mm_loader = DataLoader(train_mm_ds, batch_size=32, sampler=sampler, num_workers=0, collate_fn=multimodal_collate_fn)
val_mm_loader = DataLoader(val_mm_ds, batch_size=32, shuffle=False, num_workers=0, collate_fn=multimodal_collate_fn)
test_mm_loader = DataLoader(test_mm_ds, batch_size=32, shuffle=False, num_workers=0, collate_fn=multimodal_collate_fn)

model4 = MultiModalFusion(n_features=len(feature_cols), audio_length=3200, num_classes=2, fusion_type='late').to(device)
criterion = nn.CrossEntropyLoss(weight=class_weights)
optimizer = torch.optim.Adam(model4.parameters(), lr=1e-4)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=5)

save_dir = OUTPUT_DIR / 'hybrid_models' / 'multimodal_fusion'
save_dir.mkdir(parents=True, exist_ok=True)

history4, best_epoch4 = train_model(model4, train_mm_loader, val_mm_loader, criterion, optimizer, scheduler,
                                    device, num_epochs=50, save_dir=save_dir, model_name='multimodal_fusion', early_stopping_patience=10)

checkpoint = torch.load(save_dir / 'best_model.pt')
model4.load_state_dict(checkpoint['model_state_dict'])
test_metrics4, _, _, _ = evaluate_model(model4, test_mm_loader, criterion, device)

with open(save_dir / 'test_metrics.json', 'w') as f:
    json.dump(test_metrics4, f, indent=2)

print(f"Model 4 Test - Acc: {test_metrics4['accuracy']:.4f}, F1: {test_metrics4['f1']:.4f}, ROC-AUC: {test_metrics4['roc_auc']:.4f}")


Epoch 1/50
--------------------------------------------------


Train Loss: 0.3220, Train Acc: 0.8133
Val Loss: 0.2335, Val Acc: 0.8926
Val F1: 0.8955, Val ROC-AUC: 0.9719
✓ New best model saved! (F1: 0.8955)

Epoch 2/50
--------------------------------------------------


Train Loss: 0.2222, Train Acc: 0.8948
Val Loss: 0.1953, Val Acc: 0.9134
Val F1: 0.9154, Val ROC-AUC: 0.9791
✓ New best model saved! (F1: 0.9154)

Epoch 3/50
--------------------------------------------------


Train Loss: 0.1968, Train Acc: 0.9070
Val Loss: 0.1868, Val Acc: 0.9207
Val F1: 0.9222, Val ROC-AUC: 0.9792
✓ New best model saved! (F1: 0.9222)

Epoch 4/50
--------------------------------------------------


Train Loss: 0.1930, Train Acc: 0.9105
Val Loss: 0.1835, Val Acc: 0.9188
Val F1: 0.9207, Val ROC-AUC: 0.9818

Epoch 5/50
--------------------------------------------------


...

Train Loss: 0.1198, Train Acc: 0.9453
Val Loss: 0.1460, Val Acc: 0.9423
Val F1: 0.9429, Val ROC-AUC: 0.9868

Epoch 24/50
--------------------------------------------------


Train Loss: 0.1252, Train Acc: 0.9421
Val Loss: 0.1498, Val Acc: 0.9413
Val F1: 0.9419, Val ROC-AUC: 0.9860

Epoch 25/50
--------------------------------------------------


Train Loss: 0.1175, Train Acc: 0.9451
Val Loss: 0.1540, Val Acc: 0.9305
Val F1: 0.9317, Val ROC-AUC: 0.9863

Epoch 26/50
--------------------------------------------------


Train Loss: 0.1135, Train Acc: 0.9494
Val Loss: 0.1514, Val Acc: 0.9423
Val F1: 0.9428, Val ROC-AUC: 0.9862

Early stopping at epoch 26
Best F1: 0.9438 at epoch 16


Model 4 Test - Acc: 0.9405, F1: 0.9410, ROC-AUC: 0.9842
